In [1]:
from streamer import NIStreamer
from utils import iplot, RendOption
import numpy as np

In [2]:
ni_stramer = NIStreamer()

In [3]:
TRIG_LINE = 'PXI_Trig0'
REF_CLK_LINE = 'PXI_Trig7'

In [4]:
ao_card_list = []
do_card_list = []

In [5]:
# AO cards
for idx, max_name in enumerate(['PXI1Slot3', 'PXI1Slot5']):
    ao_card = ni_stramer.add_ao_card(max_name=max_name, samp_rate=1e6)
    ao_card.cfg_start_trig(line=TRIG_LINE, export=True if idx==0 else False)
    ao_card.cfg_ref_clk(line=REF_CLK_LINE, export=True if idx==0 else False)
    ao_card_list.append(ao_card)
    

# MF card
mf_card = ni_stramer.add_ao_card(max_name='PXI1Slot4', samp_rate=1e6)
mf_card.cfg_start_trig(line=TRIG_LINE)
mf_card.cfg_ref_clk(line=REF_CLK_LINE)

# DO cards
for max_name in ['PXI1Slot6', 'PXI1Slot7', 'PXI1Slot8', 'PXI1Slot9']:
    do_card = ni_stramer.add_do_card(max_name=max_name, samp_rate=10e6)
    do_card.cfg_start_trig(line=TRIG_LINE)
    do_card.cfg_samp_clk_src(src=REF_CLK_LINE)
    do_card_list.append(do_card)

In [9]:
ao_bank = []
do_bank = []

for ao_card in ao_card_list:
    for chan_idx in range(8):
        ao_bank.append(
            ao_card.add_chan(chan_idx=4*chan_idx)
        )
        
for chan_idx in range(4):
    ao_bank.append(
        mf_card.add_chan(chan_idx=chan_idx)
    )
    
for do_card in do_card_list:
    for port_idx in range(4):
        for line_idx in range(8):
            do_bank.append(do_card.add_chan(
                port_idx=port_idx,
                line_idx=line_idx
            ))

In [14]:
ni_stramer.clear_edit_cache()

total_dur = 10.0
instr_dur = 10e-3
instr_num = int(total_dur // instr_dur)

# AO pulse settings
amp = 0.5
freq = 100e3
buf = 50e-6

t = 0
for idx in range(instr_num):
    
#     print(f'Adding instruction {idx} ===============')
    
    for ao_chan in ao_bank:
        ao_chan.sine(
            t=t, dur=instr_dur-buf,
            amp=amp, freq=freq
        )
        
    for do_chan in do_bank:
        do_chan.go_high(t=t)
        do_chan.go_low(t=t + instr_dur - buf)
        
    t += instr_dur

90000 ... 100000

100000-1 ... 110,000-1
(99,999       109,999)

In [17]:
iplot(
    chan_list=[ao_bank[0], do_bank[0]],
    nsamps=1.2345678e5,
    t_end=15e-3
)

In [18]:
ni_stramer.compile()

9.98995

In [20]:
ni_stramer.stream_exp()